# Fine-tune Tamil Mistral 7b instruct for Tamil Poem


In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import torch
from datasets import load_dataset,concatenate_datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

In [ ]:
!pip install -q wandb -U

import wandb, os
wandb.login()

wandb_project = "tamil-mistral-7b-instuct-test"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 85.4 MB/s eta 0:00:00


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "Hemanth-thunder/Tamil-Mistral-7B-Instruct-v0.1"

# The instruction dataset to use
dataset_names = [
    "kodebot/Thirukural_tamil_with_meaning",
    "kodebot/Agananuru_Tamil_with_meaning",
    "kodebot/Purananuru_Tamil_with_meaning"
]

# Fine-tuned model name
new_model = "tamil-mistral-7b-instruct-tamil-poem-fine-tune-experiment-v0.1"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 32

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 8

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_8bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = 500

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load dataset (you can process it here)
dataset_1 = load_dataset(dataset_names[0], split="train")
dataset_split_1 = dataset_1.train_test_split(test_size=0.1)

dataset_2 = load_dataset(dataset_names[1], split="train")
dataset_split_2 = dataset_2.train_test_split(test_size=0.1)

dataset_3 = load_dataset(dataset_names[2], split="train")
dataset_split_3 = dataset_2.train_test_split(test_size=0.1)

dataset = concatenate_datasets([dataset_1, dataset_2, dataset_3])
dataset_train = concatenate_datasets([dataset_split_1['train'], dataset_split_2['train'], dataset_split_3['train']])
dataset_test = concatenate_datasets([dataset_split_1['test'], dataset_split_2['test'], dataset_split_3['test']])

dataset_train.save_to_disk("./dataset_train")
dataset_test.save_to_disk("./dataset_test")



README.md:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

thirukural_instruction.txt:   0%|          | 0.00/675k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1330 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

aganaanuru_web_instructions.txt:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

purananuru_web_instructions.txt:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/398 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1917 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/213 [00:00<?, ? examples/s]

In [ ]:

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)



Your GPU supports bfloat16: accelerate training with bf16=True


In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:

# Load Mistral tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
     target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer_config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [ ]:

max_length = 512 # This was an appropriate max length for my dataset

def generate_and_tokenize_prompt2(prompt):
    print(prompt)
    result = tokenizer(
        # formatting_func(prompt),
        prompt['text'],
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result
tokenized_dataset_train = dataset_train.map(generate_and_tokenize_prompt2)
tokenized_dataset_test = dataset_test.map(generate_and_tokenize_prompt2)

Map:   0%|          | 0/1917 [00:00<?, ? examples/s]

{'text': '<s>[INST] கற்றில னாயினுங் கேட்க அஃதொருவற் கொற்கத்தின் ஊற்றாந் துணை [/INST] நூல்களைக் கற்றவில்லையாயினும், கற்றறிந்தவர்களிடம் கேட்டறிய வேண்டும், அது ஒருவனுக்கு வாழ்க்கையில் தளர்ச்சி வந்த போது ஊன்றுகோல் போல் துணையாகும்.</s>'}
{'text': '<s>[INST] மன்னார்க்கு மன்னுதல் செங்கோன்மை அஃதின்றேல் மன்னாவாம் மன்னர்க் கொளி [/INST] அரசர்க்கு புகழ் நிலைபெறக் காரணம் செங்கோல் முறையாகும், அஃது இல்லையானால் அரசர்க்கு புகழ் நிலைபெறாமல் போகும்.</s>'}
{'text': '<s>[INST] தெளிவி லதனைத் தொடங்கார் இளிவென்னும் ஏதப்பா டஞ்சு பவர் [/INST] இழிவு தருவதாகியக் குற்றத்திற்கு அஞ்சுகின்றவர் (இன்ன ஊதியம் பயிக்கும் என்னும்) தெளிவு இல்லாத செயலைத் தொடங்கமாட்டார்.</s>'}
{'text': '<s>[INST] இன்பம் ஒருவற் கிரத்தல் இரந்தவை துன்பம் உறாஅ வரின் [/INST] இரந்து கேட்ட பொருள் துன்பமுறாமல் கிடைக்குமானால், அவ்வாறு இரத்தலும் இன்பம் என்று சொல்லத் தக்கதாகும்.</s>'}
{'text': '<s>[INST] கேட்டினும் உண்டோர் உறுதி கிளைஞரை நீட்டி அளப்பதோர் கோல் [/INST] கேடு வந்த போதும் ஒருவகை நன்மை உண்டு, அக் கேடு ஒருவனுடைய நண்பரின் இயல்புகளை நீட்டிஅளந்து 

Map:   0%|          | 0/213 [00:00<?, ? examples/s]

{'text': '<s>[INST] ஒற்றொற் றுணராமை யாள்க உடன்மூவர் சொற்றொக்க தேறப் படும் [/INST] ஓர் ஒற்றனை மற்றோர் ஒற்றன் அறியாதபடி ஆள வேண்டும், அவ்வாறு ஆளப்பட்ட ஒற்றர் மூவரின் சொல் ஒத்திருந்தால் அவை உண்மை எனத் தெளியப்படும்.</s>'}
{'text': '<s>[INST] அகர முதல எழுத்தெல்லாம் ஆதி பகவன் முதற்றே உலகு [/INST] எழுத்துக்கள் எல்லாம் அகரத்தை அடிப்படையாக கொண்டிருக்கின்றன. அதுபோல உலகம் கடவுளை அடிப்படையாக கொண்டிருக்கிறது.</s>'}
{'text': '<s>[INST] வாள்போல் பகைவரை அஞ்சற்க அஞ்சுக கேள்போல் பகைவர் தொடர்பு [/INST] வாளைப்போல் வெளிப்படையான பகைவர்க்கு அஞ்ச வேண்டியதில்லை, ஆனால் உறவினரைப் போல் இருந்து உட்பகை கொண்டவரின் தொடர்புக்கு அஞ்ச வேண்டும்.</s>'}
{'text': '<s>[INST] மங்கலம் என்ப மனைமாட்சி மற்றதன் நன்கலம் நன்மக்கட் பேறு [/INST] மனைவியின் நற்பண்பே இல்வாழ்க்கைக்கு மங்கலம் என்று கூறுவர்: நல்ல மக்களைப் பெறுதலே அதற்கு நல்லணிகலம் என்று கூறுவர்.</s>'}
{'text': '<s>[INST] ஆள்வினையும் ஆன்ற அறிவும் எனவிரண்டின் நீள்வினையான் நீளும் குடி [/INST] முயற்சி நிறைந்த அறிவு என்று சொல்லப்படும் இரண்டினையும் உடைய இடைவிடாத செயலால் ஒருவனுடை

In [ ]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 6815744 || all params: 3911438336 || trainable%: 0.17425160297861333


In [ ]:

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=2,#num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=False,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb"
)


In [ ]:

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset_train,
    # peft_config=peft_config,
    args=training_arguments,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: Currently logged in as: kodebot (kodebot-limited). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,46.580400
50,39.606200
75,37.302800
100,36.162300
125,35.221600
150,33.777200
175,33.542300
200,32.009400
225,31.951700
250,30.848100


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

generated_texts = []
reference_texts = []
for inst in dataset_test:
  # Run text generation pipeline with our next model
  splits = inst["text"].replace("<s>", "").replace("</s>", "").replace("[INST]", "").split("[/INST]")
  prompt = splits[0].strip()
  reference_texts.append(splits[1].strip())
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
  result = pipe(f"<s>[INST] {prompt} [/INST]")
  generated_texts.append(result[0]['generated_text'].replace("<s>", "").replace("</s>", "").replace("[INST]", "").split("[/INST]")[1])
  print(len(generated_texts), " of ", 10)
  if len(generated_texts) >= 10:
    break

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


1  of  10
2  of  10
3  of  10
4  of  10
5  of  10
6  of  10
7  of  10
8  of  10
9  of  10
10  of  10


In [ ]:
reference_texts

['ஓர் ஒற்றனை மற்றோர் ஒற்றன் அறியாதபடி ஆள வேண்டும், அவ்வாறு ஆளப்பட்ட ஒற்றர் மூவரின் சொல் ஒத்திருந்தால் அவை உண்மை எனத் தெளியப்படும்.',
 'எழுத்துக்கள் எல்லாம் அகரத்தை அடிப்படையாக கொண்டிருக்கின்றன. அதுபோல உலகம் கடவுளை அடிப்படையாக கொண்டிருக்கிறது.',
 'வாளைப்போல் வெளிப்படையான பகைவர்க்கு அஞ்ச வேண்டியதில்லை, ஆனால் உறவினரைப் போல் இருந்து உட்பகை கொண்டவரின் தொடர்புக்கு அஞ்ச வேண்டும்.',
 'மனைவியின் நற்பண்பே இல்வாழ்க்கைக்கு மங்கலம் என்று கூறுவர்: நல்ல மக்களைப் பெறுதலே அதற்கு நல்லணிகலம் என்று கூறுவர்.',
 'முயற்சி நிறைந்த அறிவு என்று சொல்லப்படும் இரண்டினையும் உடைய இடைவிடாத செயலால் ஒருவனுடைய குடி உயர்ந்து விளங்கும்.',
 'பெண்தன்மை உடைய இந்தப் பேதைக்குக் கண்கள் கண்டவரின் உயிரை உண்ணும் தோற்றத்தோடு கூடி ஒன்றோடொன்று மாறுபட்டிருந்தன.',
 'கல்லாதவர் உயிரோடிருக்கின்றனர் என்று சொல்லப்படும் அளவினரே அல்லாமல் ஒன்றும் விளையாத களர் நிலத்திற்கு ஒப்பாவர்.',
 'நீதியையும் நன்மையையும் விரும்பிப் பிறர்க்குப் பயன்பட வாழும் பெரியோரின் நல்லப் பண்பை உலகத்தார் போற்றிக் கொண்டாடுவர்.',
 'சால்புக்கு உரைகல் போல் மதிப்பிடும் கருவ

In [ ]:
generated_texts

['',
 '',
 ' சரியான பதிலுடன் வேலையை வெற்றிகரமாக முடிக்க. தேவையான தகவலை உள்ளிடவும்.\n\n\\### பதிலுக்கு, ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்போல் ஔிவிற்ப',
 '',
 '',
 '',
 ' சரியான பதிலுடன் வேலையை வெற்றிகரமாக முடிக்க. தேவையான தகவலை உள்ளிடவும்.\n\n\\### பதில்:\nகடல் கடந்து செல்லும் கப்பலைக் கண்ட தலைவன் தன் நாட்டுக்குத் திரும்பச் செல்ல வேண்டும் என்று நினைக்கிறான். அவன் தன் நாட்டிற்குத் திரும்பச் செல்ல வேண்டும் என்று நினைக்கக் காரணம் என்ன? அவன் தன் நாட்டிற்குத் திரும்பச் செல்ல வேண்டும் என்று நினைக்கக் காரணம் என்ன? அவன் தன் நாட்டிற்குத் திரும்பச் செல்ல வேண்டும் என்று நினைக்கக் காரணம் என்ன? அவன் தன் நாட்டிற்குத் திரும்பச் செல்ல வேண்டும் என்று நினைக்கக் காரணம் என்ன? அவன் தன் நாட்டிற்குத் திரும்பச் செல்ல வேண்டும் என்று நினைக்கக் காரணம் என்ன? அவன் தன் நாட்டிற்குத் த

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install evaluate rouge_score

In [ ]:
import evaluate

In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=generated_texts,
                         references=reference_texts)
print(results)

In [ ]:
# Empty VRAM
del model
del pipe
del trainer


In [ ]:

import gc
gc.collect()
gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)